In [31]:
from __future__ import print_function
__doc__ = 'Run all Classification algorithm with parameter search'
print(__doc__)

# IMPORT UTILITY FUNCTIONS
import logging
import numpy as np
import pandas as pd
import csv
from pprint import pprint
import matplotlib.pyplot as plt
from pylab import savefig
import pickle
from  datetime import datetime
import sys
from sklearn.datasets import load_boston
# from configobj import ConfigObj
# from __future__ import print_function

# IMPORT STATISTICAL FUNCTIONS
from scipy.stats import zscore
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
import sklearn.metrics as pm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

#import non_categorised functions
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

Run all Classification algorithm with parameter search


In [37]:
home_dir = '/home/manavee/' #user home
ws = 'dump_R1/data_master/'   #workspace
data_dir  = 'bank_marketing_campaign/bank/'  #data directory
file_list_1 = ['bank.csv', 'bank-full.csv', 'bank-names.txt']

f0 = home_dir+ws+data_dir+file_list_1[0]
f1 = home_dir+ws+data_dir+file_list_1[1]
f2 = home_dir+ws+data_dir+file_list_1[2]
df = pd.read_csv(f0,sep = ';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [48]:
def columntype(x):
    t = ''
    if is_numeric_dtype(x) == True:
        t = 'numeric'
    else:
        t = 'string'
    
    return t

def unique_value_counts(x,y):
    if y <= 10:
        return (s2[x].value_counts()).to_dict()

In [127]:
df = pd.read_csv(f0,sep = ';')
t = pd.DataFrame(df.columns,columns=['col_name'])

# def col_summary(col_name,df, p1 = ''): This is what i want to do
def col_summary(col_name): #this i am doing currently
    """summarise all cols"""
    col = df[col_name]
    TotalRows = len(df)
    TotalUniqueLevels = col.nunique()
    PercentUniqueValues = TotalUniqueLevels/TotalRows
    TotalNullValues = col.isnull().sum()
    PercentNullValues = TotalNullValues/TotalRows
    IdColFlag = TotalRows==TotalUniqueLevels
    DataType = columntype(col)
    UniqueValue = col.value_counts().to_dict() if TotalUniqueLevels <= 10 else {}
    mean = round(col.mean(),2) if DataType == 'numeric' else None
    min = round(col.min(),2) if DataType == 'numeric' else None
    q1 = round(col.quantile(q=0.25),2) if DataType == 'numeric' else None
    median = round(col.median(),2) if DataType == 'numeric' else None
    q3 = round(col.quantile(q=0.75),2) if DataType == 'numeric' else None
    max = round(col.max(),2) if DataType == 'numeric' else None
    null_flag = col.isnull().all()
    constant_flag = TotalUniqueLevels==1
    output = [TotalRows,TotalUniqueLevels,PercentUniqueValues,TotalNullValues, PercentNullValues,IdColFlag,\
              DataType,UniqueValue,min,q1,mean,median,q3,max,null_flag,constant_flag]
    return output

# defining all these cols look undity, how to improve?  to dict maybe?
t['TotalRows'],t['TotalUniqueLevels'],t['PercentUniqueValues'],t['TotalNullValues'],t['PercentNullValues']\
,t['IdColFlag'],t['DataType'],t['UniqueValue'],t['Min'],t['Q1'],t['Mean'],t['Median'],t['Q3'],t['Max']\
,t['Null_flag'],t['Constant_flag'] = zip(*t['col_name'].map(col_summary))  #pass multiple params to map
print(t)

,col_name,TotalRows,TotalUniqueLevels,PercentUniqueValues,TotalNullValues,PercentNullValues,IdColFlag,DataType,UniqueValue,Min,Q1,Mean,Median,Q3,Max,Null_flag,Constant_flag
0,age,4521,67,0.014820,0,0.0,False,numeric,{},19,33,41.17,39,49,87,False,False
1,job,4521,12,0.002654,0,0.0,False,string,{},None,None,None,None,None,None,False,False
2,marital,4521,3,0.000664,0,0.0,False,string,"{'married': 2797, 'single': 1196, 'divorced': ...",None,None,None,None,None,None,False,False
3,education,4521,4,0.000885,0,0.0,False,string,"{'secondary': 2306, 'tertiary': 1350, 'primary...",None,None,None,None,None,None,False,False
4,default,4521,2,0.000442,0,0.0,False,string,"{'no': 4445, 'yes': 76}",None,None,None,None,None,None,False,False
5,balance,4521,2353,0.520460,0,0.0,False,numeric,{},-3313,69,1422.66,444,1480,71188,False,False
6,housing,4521,2,0.000442,0,0.0,False,string,"{'yes': 2559, 'no': 1962}",None,None,None,None,None,None,False,False
7,loan,4521,2,0.000442,0,0.0,False,string,"{'no': 3830, 'yes': 691}",None,None,None,None,None,None,False,False
8,contact,4521,3,0.000664,0,0.0,False,string,"{'cellular': 2896, 'unknown': 1324, 'telephone...",None,None,None,None,None,None,False,False
9,day,4521,31,0.006857,0,0.0,False,numeric,{},1,9,15.92,16,21,31,False,False


In [112]:
t = pd.DataFrame([[i] for i in range(10)], columns=['num'])
def powers(x):
    return [x, x**2, x**3, x**4, x**5, x**6,None,234,'jdhdfs']

t['p1'], t['p2'], t['p3'], t['p4'], t['p5'], t['p6'],t['none_col'] , t['const_int'], t['const_string']= zip(*t['num'].map(powers))

print(t)
for col in t.columns[:]:
    print(col_summary(t,col))


   num  p1  p2   p3    p4     p5      p6 none_col  const_int const_string
0    0   0   0    0     0      0       0     None        234       jdhdfs
1    1   1   1    1     1      1       1     None        234       jdhdfs
2    2   2   4    8    16     32      64     None        234       jdhdfs
3    3   3   9   27    81    243     729     None        234       jdhdfs
4    4   4  16   64   256   1024    4096     None        234       jdhdfs
5    5   5  25  125   625   3125   15625     None        234       jdhdfs
6    6   6  36  216  1296   7776   46656     None        234       jdhdfs
7    7   7  49  343  2401  16807  117649     None        234       jdhdfs
8    8   8  64  512  4096  32768  262144     None        234       jdhdfs
9    9   9  81  729  6561  59049  531441     None        234       jdhdfs
[10, 10, 1.0, 0, 0.0, True, 'numeric', {9: 1, 8: 1, 7: 1, 6: 1, 5: 1, 4: 1, 3: 1, 2: 1, 1: 1, 0: 1}, 0, 2.25, 4.5, 4.5, 6.75, 9, False, False]
[10, 10, 1.0, 0, 0.0, True, 'numeric', {9: 

In [ ]:
x = 

In [79]:
print(type(df.describe().T))
df.describe().T

<class 'pandas.core.frame.DataFrame'>


,count,mean,std,min,25%,50%,75%,max
age,4521.0,41.170095,10.576211,19.0,33.0,39.0,49.0,87.0
balance,4521.0,1422.657819,3009.638142,-3313.0,69.0,444.0,1480.0,71188.0
day,4521.0,15.915284,8.247667,1.0,9.0,16.0,21.0,31.0
duration,4521.0,263.961292,259.856633,4.0,104.0,185.0,329.0,3025.0
campaign,4521.0,2.793630,3.109807,1.0,1.0,2.0,3.0,50.0
pdays,4521.0,39.766645,100.121124,-1.0,-1.0,-1.0,-1.0,871.0
previous,4521.0,0.542579,1.693562,0.0,0.0,0.0,0.0,25.0


pandas.core.frame.DataFrame

In [32]:

cmd = pd.DataFrame(df.columns,columns=['column_name'])

cmd['IdColFlag'] = cmd['TotalRows'] == cmd['TotalUniqueLevels']
cmd['DataType'] = cmd['column_name'].apply(lambda col:columntype(df[col]))
cmd['UniqueValue'] = cmd[cmd['TotalUniqueLevels'] < 11]['column_name'].apply(lambda col:(df[col].value_counts()).to_dict())
cmd['mean'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(df[col].mean(),2))
cmd['min'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(df[col].min(),2))
cmd['q1'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(df[col].quantile(q=0.25),2))
cmd['median'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(df[col].median(),2))
cmd['q3'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(df[col].quantile(q=0.75),2))
cmd['max'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(df[col].max(),2))


NullColumnList = list(cmd[cmd['TotalNullValues'] == cmd['TotalRows']]['column_name'])
FixedColumns = list(cmd[cmd['TotalUniqueLevels'] == 1][cmd['TotalNullValues'] == 0]['column_name'])



/home/manavee/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,column_name,TotalRows,TotalUniqueLevels,PercentUniqueValues,TotalNullValues,PercentNullValues,IdColFlag,DataType,UniqueValue,mean,min,q1,median,q3,max
0,age,4521,67,0.014820,0,0.0,False,numeric,NaN,41.17,19.0,33.0,39.0,49.0,87.0
1,job,4521,12,0.002654,0,0.0,False,string,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,marital,4521,3,0.000664,0,0.0,False,string,"{'married': 2797, 'single': 1196, 'divorced': ...",NaN,NaN,NaN,NaN,NaN,NaN
3,education,4521,4,0.000885,0,0.0,False,string,"{'secondary': 2306, 'tertiary': 1350, 'primary...",NaN,NaN,NaN,NaN,NaN,NaN
4,default,4521,2,0.000442,0,0.0,False,string,"{'no': 4445, 'yes': 76}",NaN,NaN,NaN,NaN,NaN,NaN
5,balance,4521,2353,0.520460,0,0.0,False,numeric,NaN,1422.66,-3313.0,69.0,444.0,1480.0,71188.0
6,housing,4521,2,0.000442,0,0.0,False,string,"{'yes': 2559, 'no': 1962}",NaN,NaN,NaN,NaN,NaN,NaN
7,loan,4521,2,0.000442,0,0.0,False,string,"{'no': 3830, 'yes': 691}",NaN,NaN,NaN,NaN,NaN,NaN
8,contact,4521,3,0.000664,0,0.0,False,string,"{'cellular': 2896, 'unknown': 1324, 'telephone...",NaN,NaN,NaN,NaN,NaN,NaN
9,day,4521,31,0.006857,0,0.0,False,numeric,NaN,15.92,1.0,9.0,16.0,21.0,31.0


In [ ]:
def overview(data,first_n_cols = None,log_file=False):
    """ save/display data summary  """

    attributes = data.columns
    if first_n_cols != None:
        data = data[attributes[:first_n_cols]]
    print('\n feature : shape '+ str(data.shape))
    print('\n feature : first_n_rows '+ str(data.head()))
    print('\n feature : attributes ' + str(attributes))
    print('\n feature : data_description ' + str())
#     print('\n feature : data_information ' ,data.info)
    print('\n feature : overview end')

,count,mean,std,min,25%,50%,75%,max
age,4521.0,41.170095,10.576211,19.0,33.0,39.0,49.0,87.0
balance,4521.0,1422.657819,3009.638142,-3313.0,69.0,444.0,1480.0,71188.0
day,4521.0,15.915284,8.247667,1.0,9.0,16.0,21.0,31.0
duration,4521.0,263.961292,259.856633,4.0,104.0,185.0,329.0,3025.0
campaign,4521.0,2.793630,3.109807,1.0,1.0,2.0,3.0,50.0
pdays,4521.0,39.766645,100.121124,-1.0,-1.0,-1.0,-1.0,871.0
previous,4521.0,0.542579,1.693562,0.0,0.0,0.0,0.0,25.0


In [22]:
overview(df)


 feature : shape (4521, 17)

 feature : first_n_rows    age          job  marital  education default  balance housing loan  \
0   30   unemployed  married    primary      no     1787      no   no   
1   33     services  married  secondary      no     4789     yes  yes   
2   35   management   single   tertiary      no     1350     yes   no   
3   30   management  married   tertiary      no     1476     yes  yes   
4   59  blue-collar  married  secondary      no        0     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome   y  
0  cellular   19   oct        79         1     -1         0  unknown  no  
1  cellular   11   may       220         1    339         4  failure  no  
2  cellular   16   apr       185         1    330         1  failure  no  
3   unknown    3   jun       199         4     -1         0  unknown  no  
4   unknown    5   may       226         1     -1         0  unknown  no  

 feature : attributes Index(['age', 'job', 'marital', 'e